# Advanced Tool Calling

In [1]:
import langchain_core.tools
import pydantic

import sys
sys.path.append('..')

import simplechatbot
from simplechatbot.openai import OpenAIChatBot

In [2]:
def example_tool_factory(chatbot: simplechatbot.ChatBot) -> list[langchain_core.tools.BaseTool]:
    class StoryOutline(pydantic.BaseModel):
        """Outline of the story."""
        title: str = pydantic.Field(description="Title of the story.")
        story_topic: str = pydantic.Field(description="The topic of the story.")
        narrative_arc: str = pydantic.Field(description="The narrative arc of the story.")

        ch1_title: str = pydantic.Field(description="Title of chapter 1 of the story.")
        ch1_description: str = pydantic.Field(description="Longer description of chapter 1.")

        ch2_title: str = pydantic.Field(description="Title of chapter 2 of the story.")
        ch2_description: str = pydantic.Field(description="Longer description of chapter 2.")

        ch3_title: str = pydantic.Field(description="Title of chapter 3 of the story.")
        ch3_description: str = pydantic.Field(description="Longer description of chapter 3.")

        def as_str(self) -> str:
            return f'title: {self.title}\n\ntopic: {self.story_topic}\n\nnarrative: {self.narrative_arc}\n\n(ch. 1) {self.ch1_title}: {self.ch1_description}\n\n(ch. 2) {self.ch2_title}: {self.ch2_description}\n\n(ch. 3) {self.ch3_title}: {self.ch3_description}'


    system_prompt = """
    You are designed to produce an outline of the story given some basic information from the user. 
    The outline includes a title, story topic, and narrative arc, as well as titles and descriptions 
    of three chapters. You are to take creative liberties in generating the outline. Each chapter should
    have its own narrative arc that contributes to the overall story arc. The plot should show character 
    growth and reflect very Progressive American values.
    """

    outline_structbot = chatbot.structbot_from_model(output_structure=StoryOutline, system_prompt=system_prompt)

    @langchain_core.tools.tool
    def story_outline_tool(description_of_story: str) -> StoryOutline:
        '''Create an outline of a story given a description of the story.'''
        return outline_structbot.invoke(description_of_story)

    return [story_outline_tool]

In [3]:
keychain = simplechatbot.APIKeyChain.from_json_file('../keys.json')

system_prompt = """
You are a regular chatbot with the capability to generate a story outline if the user requests.
ONLY do this if the user requests though.
"""

chatbot = OpenAIChatBot.new(
    model_name = 'gpt-4o-mini', 
    api_key=keychain['openai'],
    system_prompt=system_prompt,
    tool_factories=[example_tool_factory],
)

stream = chatbot.chat_stream(f'Create an outline for a story about two friends that have a lot of fun together.')
result = stream.print_and_collect()
result.tool_calls[0]

ToolCallInfo(id='call_riWwQuVHvN8z7D7QokPdrOUy', name='story_outline_tool', type='tool_call', args={'description_of_story': 'The story follows two friends, Alex and Jamie, who have been inseparable since childhood. They share a unique bond, filled with laughter, adventure, and a little bit of mischief. The plot revolves around a series of fun-filled days where they explore their town, embark on spontaneous adventures, and engage in playful competitions. Each chapter highlights a different escapade, from a treasure hunt in the local park to a cooking challenge at home, showcasing their friendship and the joy of living in the moment. As the story unfolds, they face a small conflict that tests their friendship, but ultimately, they learn the importance of communication and support, solidifying their bond even further. The story ends with a grand adventure that solidifies their friendship and leaves them with unforgettable memories.'}, tool=StructuredTool(name='story_outline_tool', descrip

In [4]:
outline = result.execute_tools()['story_outline_tool'].return_value
print(outline.as_str())

title: Unbreakable Bonds

topic: A celebration of friendship and adventure between two inseparable friends, Alex and Jamie.

narrative: Two friends navigate the joys and challenges of their unique bond through a series of fun-filled escapades, ultimately learning the value of communication and support, leading to a grand adventure that cements their friendship.

(ch. 1) The Treasure Hunt: Alex and Jamie discover an old map in the attic that leads to a treasure hidden in their local park. Excited by the adventure, they gather supplies and set off on a quest filled with clues, laughter, and unexpected challenges. As they navigate through the park, they reminisce about their childhood memories and deepen their bond, only to face a moment of disagreement about how to interpret the map, testing their communication skills.

(ch. 2) The Cooking Challenge: Inspired by their treasure hunt, Alex and Jamie decide to host a cooking challenge at home. They split into teams and compete to create the